In [ ]:
# 설치
!pip install -qU git+https://github.com/huggingface/transformers
!pip install -qU qwen-vl-utils langchain langchain_google_genai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 64.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
!nvidia-smi

Tue May 20 11:56:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model_id = "Qwen/Qwen2-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id, torch_dtype="auto", device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_id)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/content/info4.png",
            },
            {"type": "text", "text": "Extract texts in markdown format"},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/info4.png'

In [ ]:
print(output_text[0])

```markdown
주민등록증
홍길동(洪吉洞)
800101-2345678
서울특별시 가산디지털1로
(대릉테크노타운 18차)
2020.08.16.
서울특별시 금천구청장
```


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, root_validator
from typing import Optional
import os
from google.colab import userdata


# 구글 인증키
os.environ["GOOGLE_API_KEY"] = userdata.get('secretName')

class TransactionInfo(BaseModel):
    name: str = Field(description="이름")
    ssn_front: str = Field(description="주민등록번호 앞자리 (생년월일 6자리)")
    address: str = Field(description="주소")
    ssn_back_first: str = Field(description="주민등록번호 뒷자리의 첫 번째 숫자")
    gender: Optional[str] = Field(default=None, description="성별 (남성 또는 여성)")

    @root_validator(pre=True)
    def set_gender(cls, values):
        ssn_back = values.get('ssn_back_first')
        if ssn_back in ['1', '3']:
            values['gender'] = '남성'
        elif ssn_back in ['2', '4']:
            values['gender'] = '여성'
        else:
            values['gender'] = '알 수 없음'
        return values


# Google Gemini 모델 초기화
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm_structured_output = llm.with_structured_output(TransactionInfo)

# 체인 실행
response = llm_structured_output.invoke(output_text[0])

# 결과 출력
print(response)


<ipython-input-18-11489237d30b>:36: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @root_validator(pre=True)


name='홍길동' ssn_front='800101' address='서울특별시 가산디지털1로 (대릉테크노타운 18차)' ssn_back_first='2' gender='여성'


In [ ]:
response.name

'홍길동'

In [ ]:
dict(response)

{'name': '홍길동',
 'ssn_front': '800101',
 'address': '서울특별시 가산디지털1로 (대릉테크노타운 18차)',
 'ssn_back_first': '2',
 'gender': '여성'}